In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.impute import KNNImputer
import statsmodels.formula.api as smf
import statsmodels.api as sm

DATA_FOLDER = 'data/'
HOUSEHOLD_DATASET = DATA_FOLDER+'PisoFirme_AEJPol-20070024_household.dta'
INDIVIDUAL_DATASET = DATA_FOLDER+'PisoFirme_AEJPol-20070024_individual.dta'

household = pd.read_stata(HOUSEHOLD_DATASET)
individual = pd.read_stata(INDIVIDUAL_DATASET)

In [2]:
household.head(7)

,dpisofirme,idcluster,coord_x,coord_y,idmun,idmza,C_blocksdirtfloor,C_HHdirtfloor,C_child05,C_households,...,S_cesds,S_pss,S_instcement,S_instsanita,S_restsanita,S_constceili,S_restowalls,S_improveany,S_logrent,S_logsell
0,0.0,70000537.0,-103.50367,25.583067,7.0,40,0.3,0.036629,0.555554,819.0,...,14.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,5.298317,9.903487
1,0.0,70000537.0,-103.50367,25.583067,7.0,40,0.3,0.036629,0.555554,819.0,...,17.0,24.0,0.0,0.0,0.0,0.0,0.0,0.0,5.298317,9.615806
2,0.0,70000537.0,-103.50367,25.583067,7.0,40,0.3,0.036629,0.555554,819.0,...,16.0,16.0,0.0,0.0,0.0,0.0,0.0,0.0,6.214608,10.819778
3,0.0,70000537.0,-103.50367,25.583067,7.0,47,0.3,0.036629,0.555554,819.0,...,20.0,19.0,0.0,0.0,0.0,0.0,0.0,0.0,11.385092,11.918390
4,0.0,70000537.0,-103.50367,25.583067,7.0,47,0.3,0.036629,0.555554,819.0,...,4.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,5.703783,10.819778
5,0.0,70000537.0,-103.50367,25.583067,7.0,41,0.3,0.036629,0.555554,819.0,...,25.0,26.0,0.0,0.0,0.0,0.0,0.0,0.0,4.605170,8.517193
6,0.0,70000537.0,-103.50367,25.583067,7.0,40,0.3,0.036629,0.555554,819.0,...,18.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,5.857933,9.615806


In [16]:
individual.head(20)

,dpisofirme,idcluster,coord_x,coord_y,idmun,idmza,S_age,S_gender,S_childma,S_childmaage,...,S_milkprogram,S_foodprogram,S_seguropopular,S_hasanimals,S_animalsinside,S_garbage,S_washhands,S_incomepc,S_cashtransfers,S_assetspc
0,0.0,70000537.0,-103.503670,25.583067,7.0,47,1.0,0,1.0,27.0,...,0.0,0.0,0.0,1.0,0.0,1.0,2.0,0.000000,0.0,21158.214844
1,0.0,70000537.0,-103.503670,25.583067,7.0,40,37.0,0,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,1.0,3.0,1000.000000,0.0,53054.988281
2,0.0,70000537.0,-103.503670,25.583067,7.0,40,1.0,0,1.0,18.0,...,0.0,0.0,0.0,0.0,0.0,1.0,5.0,1100.000000,0.0,52930.226562
3,0.0,70000537.0,-103.503670,25.583067,7.0,47,43.0,1,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,1.0,4.0,660.416687,0.0,30632.578125
4,0.0,70000537.0,-103.503670,25.583067,7.0,47,4.0,1,1.0,27.0,...,0.0,0.0,0.0,1.0,0.0,1.0,2.0,0.000000,0.0,21158.214844
5,0.0,70000537.0,-103.503670,25.583067,7.0,40,4.0,1,1.0,30.0,...,0.0,0.0,0.0,0.0,0.0,1.0,3.0,560.000000,60.0,21200.611328
6,0.0,70000537.0,-103.503670,25.583067,7.0,40,18.0,0,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,1.0,5.0,1100.000000,0.0,52930.226562
7,0.0,70000537.0,-103.503670,25.583067,7.0,40,3.0,0,0.0,NaN,...,0.0,0.0,0.0,1.0,0.0,1.0,3.0,1108.333374,0.0,35087.679688
8,0.0,70000537.0,-103.503670,25.583067,7.0,47,5.0,1,1.0,30.0,...,0.0,0.0,0.0,0.0,0.0,1.0,4.0,660.416687,0.0,30632.578125
9,0.0,70000537.0,-103.503670,25.583067,7.0,40,30.0,0,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,1.0,3.0,560.000000,60.0,21200.611328


In [6]:
temp = individual['S_childmaage'].copy()
msk = individual['S_childma'].apply(lambda x : True if x == 0 else False)
temp[msk] = 0
individual['S_childmaage'] = temp

In [8]:
temp = individual['S_childmaeduc'].copy()
msk = individual['S_childma'].apply(lambda x : True if x == 0 else False)
temp[msk] = 0
individual['S_childmaeduc'] = temp

In [10]:
temp = individual['S_childpaage'].copy()
msk = individual['S_childpa'].apply(lambda x : True if x == 0 else False)
temp[msk] = 0
individual['S_childpaage'] = temp

In [11]:
temp = individual['S_childpaeduc'].copy()
msk = individual['S_childpa'].apply(lambda x : True if x == 0 else False)
temp[msk] = 0
individual['S_childpaeduc'] = temp

# Data processing

### A few stats

In [12]:
print("We have ",len(household.index)," households.")
print("We have ", len(household.columns)," variables for households.")
print("How many missing values is there ? ", household.isna().any().sum())

We have  2783  households.
We have  78  variables for households.
How many missing values is there ?  51


In [13]:
print("We have ",len(individual.index)," individuals.")
print("We have ", len(individual.columns)," variables for individuals.")
print("How many missing values is there ? ", individual.isna().any().sum())

We have  6693  individuals.
We have  89  variables for individuals.
How many missing values is there ?  24


### Missing values

In [14]:
individual['S_gender'] = individual['S_gender'].apply(lambda x : 1 if type(x) == str else 0)

In [15]:
#Missing values will be imputed with a K-Nearest Neigbors algorithm
imputer = KNNImputer()

#Households
mod = imputer.fit(household)
H = pd.DataFrame(mod.transform(household))
H.columns = household.columns

#Individuals
temp = imputer.fit(individual)
I = pd.DataFrame(temp.transform(individual))
I.columns = individual.columns

In [16]:
H.head()

,dpisofirme,idcluster,coord_x,coord_y,idmun,idmza,C_blocksdirtfloor,C_HHdirtfloor,C_child05,C_households,...,S_cesds,S_pss,S_instcement,S_instsanita,S_restsanita,S_constceili,S_restowalls,S_improveany,S_logrent,S_logsell
0,0.0,70000537.0,-103.50367,25.583067,7.0,40.0,0.3,0.036629,0.555554,819.0,...,14.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,5.298317,9.903487
1,0.0,70000537.0,-103.50367,25.583067,7.0,40.0,0.3,0.036629,0.555554,819.0,...,17.0,24.0,0.0,0.0,0.0,0.0,0.0,0.0,5.298317,9.615806
2,0.0,70000537.0,-103.50367,25.583067,7.0,40.0,0.3,0.036629,0.555554,819.0,...,16.0,16.0,0.0,0.0,0.0,0.0,0.0,0.0,6.214608,10.819778
3,0.0,70000537.0,-103.50367,25.583067,7.0,47.0,0.3,0.036629,0.555554,819.0,...,20.0,19.0,0.0,0.0,0.0,0.0,0.0,0.0,11.385092,11.918390
4,0.0,70000537.0,-103.50367,25.583067,7.0,47.0,0.3,0.036629,0.555554,819.0,...,4.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,5.703783,10.819778


In [17]:
I.head()

,dpisofirme,idcluster,coord_x,coord_y,idmun,idmza,S_age,S_gender,S_childma,S_childmaage,...,S_milkprogram,S_foodprogram,S_seguropopular,S_hasanimals,S_animalsinside,S_garbage,S_washhands,S_incomepc,S_cashtransfers,S_assetspc
0,0.0,70000537.0,-103.50367,25.583067,7.0,47.0,1.0,0.0,1.0,27.0,...,0.0,0.0,0.0,1.0,0.0,1.0,2.0,0.000000,0.0,21158.214844
1,0.0,70000537.0,-103.50367,25.583067,7.0,40.0,37.0,0.0,0.8,19.2,...,0.0,0.0,0.0,0.0,0.0,1.0,3.0,1000.000000,0.0,53054.988281
2,0.0,70000537.0,-103.50367,25.583067,7.0,40.0,1.0,0.0,1.0,18.0,...,0.0,0.0,0.0,0.0,0.0,1.0,5.0,1100.000000,0.0,52930.226562
3,0.0,70000537.0,-103.50367,25.583067,7.0,47.0,43.0,1.0,1.0,26.6,...,0.0,0.0,0.0,0.0,0.0,1.0,4.0,660.416687,0.0,30632.578125
4,0.0,70000537.0,-103.50367,25.583067,7.0,47.0,4.0,1.0,1.0,27.0,...,0.0,0.0,0.0,1.0,0.0,1.0,2.0,0.000000,0.0,21158.214844


In [18]:
print("How many missing values is there ? ", H.isna().any().sum())
print("How many missing values is there ? ", I.isna().any().sum())

How many missing values is there ?  0
How many missing values is there ?  0


In [19]:
#No more missing values in the dataframes but we need to adjust some values because the KNN returns decimal numbers
#and features are categorical they only can be 0 or 1. We will take 1 if the value is above 0.5 and 0 otherwise. 
house_f = H.drop(['dpisofirme', 'idcluster', 'coord_x', 'coord_y', 'idmun', 'idmza', 'C_blocksdirtfloor', 
                  'C_HHdirtfloor', 'C_child05', 'C_rooms', 'C_HHpersons', 'C_waterland', 'C_waterhouse',
                  'C_waterbath', 'C_gasheater', 'C_refrigerator', 'C_washing', 'C_telephone', 'C_vehicle',
                  'C_overcrowding', 'C_poverty', 'C_illiterate', 'C_headeduc', 'C_dropouts515', 'C_employment',
                  'C_earnincome', 'S_cementfloor2000', 'S_incomepc', 'S_assetspc', 'S_shpeoplework', 'S_hrsworkedpc',
                  'S_consumptionpc', 'S_cashtransfers', 'S_dem1', 'S_dem2', 'S_dem3', 'S_dem4', 'S_dem5', 'S_dem6',
                  'S_dem7', 'S_dem8', 'S_shcementfloor', 'S_cementfloorkit', 'S_cementfloordin', 'S_cementfloorbat',
                  'S_cementfloorbed', 'S_logrent', 'S_logsell'], axis = 1).columns
indiv_f = I.drop(['dpisofirme', 'idcluster', 'coord_x', 'coord_y', 'idmun', 'idmza', 'S_haz', 'S_whz', 'S_malincom',
                 'S_palincom', 'S_incomepc', 'S_cashtransfers', 'S_assetspc'], axis = 1).columns

for f in house_f:
    H[f] = H[f].apply(lambda x : int(x) if abs(int(x)-x)<0.5 else int(x+1))
    
for f in indiv_f:
    I[f] = I[f].apply(lambda x : int(x) if abs(int(x)-x)<0.5 else int(x+1))

In [20]:
H.head()

,dpisofirme,idcluster,coord_x,coord_y,idmun,idmza,C_blocksdirtfloor,C_HHdirtfloor,C_child05,C_households,...,S_cesds,S_pss,S_instcement,S_instsanita,S_restsanita,S_constceili,S_restowalls,S_improveany,S_logrent,S_logsell
0,0.0,70000537.0,-103.50367,25.583067,7.0,40.0,0.3,0.036629,0.555554,819,...,14,12,0,0,0,0,0,0,5.298317,9.903487
1,0.0,70000537.0,-103.50367,25.583067,7.0,40.0,0.3,0.036629,0.555554,819,...,17,24,0,0,0,0,0,0,5.298317,9.615806
2,0.0,70000537.0,-103.50367,25.583067,7.0,40.0,0.3,0.036629,0.555554,819,...,16,16,0,0,0,0,0,0,6.214608,10.819778
3,0.0,70000537.0,-103.50367,25.583067,7.0,47.0,0.3,0.036629,0.555554,819,...,20,19,0,0,0,0,0,0,11.385092,11.918390
4,0.0,70000537.0,-103.50367,25.583067,7.0,47.0,0.3,0.036629,0.555554,819,...,4,5,0,0,0,0,0,0,5.703783,10.819778


### Construction of the target

In [55]:
#We will generate a binary outcome vector indicating the global health status of the different persons. We will make 
#a combination of the different features about health.
#1 = health problems, 0 = good health
health = ['S_parcount', 'S_diarrhea', 'S_anemia', 'S_respira', 'S_skin', 'S_otherdis']

def target():
    N = len(I.index)
    target = []
    for n in range(N):
        if individual[health].loc[n].sum() >= 1:
            target.append(1)
        else:
            target.append(0)
    return target

target = pd.DataFrame(target())

### Normalization

In [52]:
to_norm = ['idcluster', 'coord_x', 'coord_y', 'idmun','idmza', 'S_age', 'S_childmaage', 'S_childmaeduc', 'S_childpaage',
          'S_childpaeduc', 'S_HHpeople', 'S_rooms', 'S_mccdts', 'S_pbdypct', 'S_haz', 'S_whz', 'S_malincom', 'S_palincom',
          'S_washhands', 'S_incomepc', 'S_cashtransfers', 'S_assetspc']
I[to_norm] = normalize(I[to_norm])

### Feature selection